# Data Processing for the recommendation model - LegacyLab

ESte documento apresenta a exploração, o tratamento e explicações de decisões no pré-processamento dos dados para a construção futura do modelo de recomendação baseado em filtragem colaborativa do LegacyLab.

## Importação da base de dados e dependências

In [1]:
!gdown 17l3SytuzJuiUf_Vyk21fEEYNRcoyHPTsyagisoPvoxg

Downloading...
From (original): https://drive.google.com/uc?id=17l3SytuzJuiUf_Vyk21fEEYNRcoyHPTsyagisoPvoxg
From (redirected): https://docs.google.com/spreadsheets/d/17l3SytuzJuiUf_Vyk21fEEYNRcoyHPTsyagisoPvoxg/export?format=xlsx
To: /content/ITL-ESM6-2024-dataset-recomendação.xlsx
276kB [00:00, 4.14MB/s]


In [2]:
import pandas as pd
caminho_excel = '/content/ITL-ESM6-2024-dataset-recomendação.xlsx'

#Dataset de usuários

## Exploração de Dados


In [3]:
df_ceo = pd.read_excel(caminho_excel)
df_ceo.head()

,id,proponente,nome da empresa,indústria de atuação,cargo
0,1,Braian Goulart,Fundação Banco do Brasil,Terceiro Setor,COO
1,2,Kerolay Ferreira,Fundação Itaú Social,Terceiro Setor,COO
2,3,Vlamir Guterres Holanda Pureza,Sesc,Cultura e Entretenimento,COO
3,4,Wilene Vila-Chã,Fundación Telefónica,Terceiro Setor,CHRO
4,5,Jamur Mantas,Acciona,Energia Renovável,COO


In [4]:
df_ceo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1424 entries, 0 to 1423
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    1424 non-null   int64 
 1   proponente            1424 non-null   object
 2   nome da empresa       1424 non-null   object
 3   indústria de atuação  1424 non-null   object
 4   cargo                 1424 non-null   object
dtypes: int64(1), object(4)
memory usage: 55.8+ KB


In [5]:
print(df_ceo.columns)

Index(['id', 'proponente', 'nome da empresa', 'indústria de atuação', 'cargo'], dtype='object')


In [6]:
def unique_values_and_counts(df, columns):
    for col in columns:
        print(f"\nContagem de valores na coluna '{col}':")
        print(df[col].value_counts())
        print("--------------------------------------------")

# Lista das colunas a serem verificadas
columns_to_check = ['proponente', 'nome da empresa', 'indústria de atuação', 'cargo']

# Chame a função para encontrar valores únicos e contar a ocorrência de cada valor em cada coluna
unique_values_and_counts(df_ceo, columns_to_check)



Contagem de valores na coluna 'proponente':
proponente
Braian Goulart                      1
Mikiko Laureano Freiria             1
Enoques Severo                      1
Warley Faro                         1
Adolf Pereira Bensaúde Cangueiro    1
                                   ..
Jardiele Simão Lira Cobra           1
Miliene Rego Bugalho Cadaval        1
Merceli Arantes                     1
Luma Carvalhais Baião               1
Damares Malho Faria                 1
Name: count, Length: 1424, dtype: int64
--------------------------------------------

Contagem de valores na coluna 'nome da empresa':
nome da empresa
Google                      313
Siemens                      91
UNESCO                       54
Tesla                        39
WWF                          39
                           ... 
Illumina                      1
Organovo                      1
Biogen                        1
Patagonia                     1
Fundação Banco do Brasil      1
Name: count, Length: 31

In [7]:
def check_username_consistency(df):
    # Agrupe o DataFrame por id e proponente e conte o número de ocorrências
    grouped = df.groupby(['id', 'proponente']).size().reset_index(name='count')

    # Filtrar as entradas onde há mais de uma proponente para o mesmo id
    inconsistent_entries = grouped[grouped.duplicated(subset='id', keep=False)]

    if inconsistent_entries.empty:
        print("Todos os usernames para cada user_id são consistentes.")
    else:
        print("Usuários com usernames inconsistentes:")
        print(inconsistent_entries)

# Chame a função para verificar a consistência do username em relação ao id
check_username_consistency(df_ceo)

Todos os usernames para cada user_id são consistentes.


## Pré-processamento

In [8]:
# Renomeando as colunas
df_ceo = df_ceo.rename(columns={'id': 'id_user', 'proponente': 'user_name', 'nome da empresa': 'enterprise', 'cargo': 'position'})
df_ceo.head()

,id_user,user_name,enterprise,indústria de atuação,position
0,1,Braian Goulart,Fundação Banco do Brasil,Terceiro Setor,COO
1,2,Kerolay Ferreira,Fundação Itaú Social,Terceiro Setor,COO
2,3,Vlamir Guterres Holanda Pureza,Sesc,Cultura e Entretenimento,COO
3,4,Wilene Vila-Chã,Fundación Telefónica,Terceiro Setor,CHRO
4,5,Jamur Mantas,Acciona,Energia Renovável,COO


In [9]:
# Removendo a coluna "indústria de atuação"
df_ceo = df_ceo.drop(columns=['indústria de atuação'])

In [10]:
df_ceo.head()

,id_user,user_name,enterprise,position
0,1,Braian Goulart,Fundação Banco do Brasil,COO
1,2,Kerolay Ferreira,Fundação Itaú Social,COO
2,3,Vlamir Guterres Holanda Pureza,Sesc,COO
3,4,Wilene Vila-Chã,Fundación Telefónica,CHRO
4,5,Jamur Mantas,Acciona,COO


## Explicações

O dataset de usuários é essencial para o desenvovimento do projeto LegacyLab. Além de popular o banco de dados (entidade usuário) vigente na solução, ele apresenta dados sobre os donos e avaliadores dos projetos, guiando também o sistema de recomendação presente na solução.

A coluna sobre a indústria de atuação foi removida pois não se encaixa no banco de dados definido pelo LegacyLab.

A coluna <industria de atuação> foi removida utilizando o seguinte código:

`df_ceo = df_ceo.drop(columns=['indústria de atuação'])`


Esse dataset final de usuários apresenta 1424 linhas de informação, fornecendo dados sobre:

| Campo         | Descrição                                         |
|---------------|---------------------------------------------------|
| `id_user`       | ID único do usuário                               |
| `user_name`         | Nome do usuário                      |
| `enterprise`          | Empresa na qual o usuário apresenta um Clevel                    |
| `position`       | Clevel do usuário

#Dataset de projetos

## Exploração de Dados

In [11]:
df_projeto = pd.read_excel(caminho_excel, sheet_name='ds-projetos')
df_projeto.head()

,id,proponente_id,projeto,setor,macrosetor,impacto-esperado,status,alcance geográfico,público-alvo
0,1,1,Como Manter a Esperança?,Psicologia,Saúde e Bem-estar,Social,Em planejamento,Local,Consumidores e Empresas
1,2,2,Labirinto do Aprendizado,Educação,Educação,Social,Em andamento,Nacional,Consumidores e Empresas
2,3,3,Energia Solar,Energia Renovável,Energia e Recursos,"Ambiental, Econômico",Em andamento,Local,Funcionários
3,4,4,CRISPR-Cas9 na Terapia Gênica,Genética,Ciência e Pesquisa,Científico,Em planejamento,Local,Funcionários
4,5,5,Campanha de Conscientização sobre Economia de ...,Sustentabilidade,Sustentabilidade,"Ambiental, Social",Em andamento,Local,Funcionários


In [12]:
df_projeto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1424 entries, 0 to 1423
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  1424 non-null   int64 
 1   proponente_id       1424 non-null   int64 
 2   projeto             1424 non-null   object
 3   setor               1424 non-null   object
 4   macrosetor          1424 non-null   object
 5   impacto-esperado    1424 non-null   object
 6   status              1424 non-null   object
 7   alcance geográfico  1424 non-null   object
 8   público-alvo        1424 non-null   object
dtypes: int64(2), object(7)
memory usage: 100.2+ KB


In [13]:
print(df_projeto.columns)

Index(['id', 'proponente_id', 'projeto', 'setor', 'macrosetor',
       'impacto-esperado', 'status', 'alcance geográfico', 'público-alvo'],
      dtype='object')


In [14]:
# Verifica contagem de itens especificos de cada coluna
def unique_values_and_counts(df, columns):
    for col in columns:
        print(f"\nContagem de valores na coluna '{col}':")
        print(df[col].value_counts())
        print("--------------------------------------------")

# Lista das colunas a serem verificadas
columns_to_check = ['proponente_id', 'projeto', 'setor', 'macrosetor',
       'impacto-esperado', 'status', 'alcance geográfico', 'público-alvo']

# Chame a função para encontrar valores únicos e contar a ocorrência de cada valor em cada coluna
unique_values_and_counts(df_projeto, columns_to_check)



Contagem de valores na coluna 'proponente_id':
proponente_id
1       1
936     1
956     1
955     1
954     1
       ..
473     1
472     1
471     1
470     1
1424    1
Name: count, Length: 1424, dtype: int64
--------------------------------------------

Contagem de valores na coluna 'projeto':
projeto
Como Manter a Esperança?                                                     1
Cidades Criativas                                                            1
Ciência Verde                                                                1
FutureTransit Hubs                                                           1
Urgências Urbanas                                                            1
                                                                            ..
AURA (Ações Unificadas para Revitalização Ambiental)                         1
TechBridge: Treinamento de Habilidades Digitais para Jovens Desempregados    1
SaúdeSegura                                                  

In [15]:
def check_username_consistency(df):
    # Agrupe o DataFrame por id e projeto e conte o número de ocorrências
    grouped = df.groupby(['id', 'projeto']).size().reset_index(name='count')

    # Filtrar as entradas onde há mais de uma projeto para o mesmo id
    inconsistent_entries = grouped[grouped.duplicated(subset='id', keep=False)]

    if inconsistent_entries.empty:
        print("Todos os projetos para cada id são consistentes.")
    else:
        print("Ids com projetos inconsistentes:")
        print(inconsistent_entries)

# Chame a função para verificar a consistência do username em relação ao user_id
check_username_consistency(df_projeto)

Todos os projetos para cada id são consistentes.


## Pré-processamento

In [16]:
df_projeto.head()

,id,proponente_id,projeto,setor,macrosetor,impacto-esperado,status,alcance geográfico,público-alvo
0,1,1,Como Manter a Esperança?,Psicologia,Saúde e Bem-estar,Social,Em planejamento,Local,Consumidores e Empresas
1,2,2,Labirinto do Aprendizado,Educação,Educação,Social,Em andamento,Nacional,Consumidores e Empresas
2,3,3,Energia Solar,Energia Renovável,Energia e Recursos,"Ambiental, Econômico",Em andamento,Local,Funcionários
3,4,4,CRISPR-Cas9 na Terapia Gênica,Genética,Ciência e Pesquisa,Científico,Em planejamento,Local,Funcionários
4,5,5,Campanha de Conscientização sobre Economia de ...,Sustentabilidade,Sustentabilidade,"Ambiental, Social",Em andamento,Local,Funcionários


In [17]:
# Renomeando as colunas
df_projeto = df_projeto.rename(columns={'id': 'id_project', 'proponente_id': 'id_owner', 'projeto': 'project_name', 'setor': 'microtheme', 'macrosetor': 'macrotheme'})
df_projeto.head()

,id_project,id_owner,project_name,microtheme,macrotheme,impacto-esperado,status,alcance geográfico,público-alvo
0,1,1,Como Manter a Esperança?,Psicologia,Saúde e Bem-estar,Social,Em planejamento,Local,Consumidores e Empresas
1,2,2,Labirinto do Aprendizado,Educação,Educação,Social,Em andamento,Nacional,Consumidores e Empresas
2,3,3,Energia Solar,Energia Renovável,Energia e Recursos,"Ambiental, Econômico",Em andamento,Local,Funcionários
3,4,4,CRISPR-Cas9 na Terapia Gênica,Genética,Ciência e Pesquisa,Científico,Em planejamento,Local,Funcionários
4,5,5,Campanha de Conscientização sobre Economia de ...,Sustentabilidade,Sustentabilidade,"Ambiental, Social",Em andamento,Local,Funcionários


In [18]:
# Removendo a coluna "indústria de atuação"
df_projeto = df_projeto.drop(columns=['impacto-esperado', 'alcance geográfico', 'público-alvo'])

In [19]:
df_projeto.head()

,id_project,id_owner,project_name,microtheme,macrotheme,status
0,1,1,Como Manter a Esperança?,Psicologia,Saúde e Bem-estar,Em planejamento
1,2,2,Labirinto do Aprendizado,Educação,Educação,Em andamento
2,3,3,Energia Solar,Energia Renovável,Energia e Recursos,Em andamento
3,4,4,CRISPR-Cas9 na Terapia Gênica,Genética,Ciência e Pesquisa,Em planejamento
4,5,5,Campanha de Conscientização sobre Economia de ...,Sustentabilidade,Sustentabilidade,Em andamento


### Macrotemas

In [20]:
# Entendendo quais os valores únicos de macrotema
df_projeto['macrotheme'].unique()

array(['Saúde e Bem-estar', 'Educação', 'Energia e Recursos',
       'Ciência e Pesquisa', 'Sustentabilidade',
       'Tecnologia e Informação', 'Comunicação e Mídia', 'Meio Ambiente',
       'Tecnologia e Inovação', 'Serviços Sociais', 'Economia e Finanças',
       'Arte e Cultura', 'Desenvolvimento Econômico e Empresarial',
       'Transporte e Logística', 'Agricultura e Alimentação',
       'Engenharia', 'Desenvolvimento Urbano e Infraestrutura',
       'Direito e Governança', 'Geral e Diversificado',
       'Saúde e Inovação', 'Gestão e Recursos Humanos',
       'Tecnologia e Comunicação', 'Saúde e Tecnologia',
       'Desenvolvimento Comunitário', 'Transporte e Tecnologia',
       'Educação e Meio Ambiente', 'Tecnologia e Dados',
       'Empreendedorismo e Inovação', 'Saúde e Indústria',
       'Desporto e Lazer', 'Estratégia e Gestão',
       'Marketing e Comunicação', 'Tecnologia e Educação',
       'Tecnologia e Meio Ambiente', 'Tecnologia e Saúde',
       'Turismo e Lazer', 'S

In [21]:
# Valores únicos de macrotema
count_unique_macrothemes = df_projeto['macrotheme'].nunique()
print("Número de macrotemas únicos:", count_unique_macrothemes)


Número de macrotemas únicos: 40


In [22]:
# Vendo quais microtemas existem por macrotema
# Agrupe os dados por macrotemas e microtemas
macro_micro_df = df_projeto.groupby(['macrotheme', 'microtheme']).size().reset_index(name='count')

# Crie um dicionário para armazenar os microtemas associados a cada macrotema
macro_micro_dict = {}

# Itere sobre cada macrotema e liste os microtemas associados
for macrotheme in macro_micro_df['macrotheme'].unique():
    microthemes = macro_micro_df[macro_micro_df['macrotheme'] == macrotheme]['microtheme'].tolist()
    macro_micro_dict[macrotheme] = microthemes

# Crie um DataFrame a partir do dicionário
macro_micro_table = pd.DataFrame.from_dict(macro_micro_dict, orient='index').transpose()

# Exiba a tabela de referência
print(macro_micro_table)


   Agricultura e Alimentação        Arte e Cultura   Ciência e Pesquisa  \
0                Agricultura           Arquitetura          Astrofísica   
1                Alimentação                  Arte           Astronomia   
2                       None        Arte e Cultura       Bioinformática   
3                       None  Arte e Meio Ambiente             Biologia   
4                       None               Cultura     Biologia Celular   
5                       None              Cultural  Biologia Estrutural   
6                       None                  None   Biologia Evolutiva   
7                       None                  None   Biologia Molecular   
8                       None                  None           Bioquímica   
9                       None                  None             Botânica   
10                      None                  None           Científico   
11                      None                  None    Ciência, Educação   
12                      N

O dataset final apresenta as informações cruciais definidas pelo grupo LegacyLab, contendo não só o id projeto e do usuário responsável, mas também a identificação daquele usuário que avaliou o projeto.


Outras informações como mirotema, macrotema, status, nome e o principal: a avaliação do usuário também se encontram disponíveis no dataset final.

Essa tabela é essencial para o modelo de recomendação! O rating presente nela guia as recomendações furutas que virão do sistema.

In [23]:
df_projeto.columns

Index(['id_project', 'id_owner', 'project_name', 'microtheme', 'macrotheme',
       'status'],
      dtype='object')

## Explicações

### Microtemas

In [24]:
# Entendendo quais os valores únicos de microtema
df_projeto['microtheme'].unique()

array(['Psicologia', 'Educação', 'Energia Renovável', 'Genética',
       'Sustentabilidade', 'TI', 'Educação, Tecnologia', 'Comunicação',
       'Ambiental', 'Tecnologia, Saúde', 'Social', 'Tecnologia',
       'Científico', 'Astronomia', 'Energia', 'Econômico',
       'Educação Cultural', 'Desenvolvimento Pessoal', 'Arte',
       'Bioquímica', 'Tecnologia, Meio Ambiente', 'Física',
       'Desenvolvimento Empresarial', 'Empreendedorismo Sustentável',
       'Cultural', 'Transporte Marítimo', 'Tecnologia, Sustentabilidade',
       'Desenvolvimento Profissional', 'Agricultura', 'Saúde',
       'Desenvolvimento Juvenil', 'Biotecnologia',
       'Conservação Ambiental', 'Cultura', 'Tecnologia, Educação',
       'Urbanismo, Sustentabilidade', 'Direitos Humanos',
       'Educação, Agricultura', 'Geral', 'Mídia', 'Filantropia',
       'Arte e Cultura', 'Transporte', 'Saúde Mental',
       'Inovação em Saúde', 'Multiculturalismo', 'Ecologia',
       'Recursos Humanos', 'Meteorologia', 'Telecom

In [25]:
# Valores únicos de microtema
count_unique_microthemes = df_projeto['microtheme'].nunique()
print("Número de microtemas únicos:", count_unique_microthemes)


Número de microtemas únicos: 203


O dataset de projetos é essencial para o desenvovimento do projeto LegacyLab. Além de popular o banco de dados (entidade projetos) vigente na solução, ele apresenta dados sobre os projetos, bem como micro e macrotema, status, entre outros.

Esse dataset guia em parte  o sistema de recomendação presente na solução, principalmente quando complementado com a tabela de avaliações.

As colunas sobre o impacto esperado, o alcance geográfico e o público-alvo foram removidas pois não se encaixam no banco de dados definido pelo LegacyLab.

Código utilizado:

```
df_projeto = df_projeto.drop(columns=['impacto-esperado', 'alcance geográfico', 'público-alvo'])```

Esse dataset finalde usuários apresenta 1424 linhas de informação, fornecendo dados sobre:

| Campo         | Descrição                                         |
|---------------|---------------------------------------------------|
| `id_project	`       | ID único do projeto                               |
| `id_owner`         | ID único do usuário responsável pelo projeto                      |
| `project_name`          | Nome do projeto                   |
| `microtheme`       | Microtema do proejto |
| `macrotheme`       | Macrotema do proejto
| `status`       | Status do proejto

**OBS: TRATAMENTO DE MICROTEMAS POR MACROTEMAS NO DATASET FINAL!**

In [26]:
df_projeto.head(1)

,id_project,id_owner,project_name,microtheme,macrotheme,status
0,1,1,Como Manter a Esperança?,Psicologia,Saúde e Bem-estar,Em planejamento


#Dataset de avaliações

## Exploração de Dados

In [27]:
# Lendo o dataset
df_avaliacao = pd.read_excel(caminho_excel, sheet_name='ds-avaliação')
df_avaliacao.head()

,id_proponente,id_projeto,avaliação
0,1,1119,4
1,1,300,3
2,1,11,4
3,1,1121,1
4,1,937,4


In [28]:
# Verificando infosmações básicas
df_avaliacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9101 entries, 0 to 9100
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   id_proponente  9101 non-null   int64
 1   id_projeto     9101 non-null   int64
 2   avaliação      9101 non-null   int64
dtypes: int64(3)
memory usage: 213.4 KB


In [29]:
# Nomes das colunass
print(df_avaliacao.columns)

Index(['id_proponente', 'id_projeto', 'avaliação'], dtype='object')


In [30]:
# Verifica quantas vezes cada id aparece (usuário, projeto e avaliação)
def unique_values_and_counts(df, columns):
    for col in columns:
        print(f"\nContagem de valores na coluna '{col}':")
        print(df[col].value_counts())
        print("--------------------------------------------")

# Lista das colunas a serem verificadas
columns_to_check = ['id_proponente', 'id_projeto', 'avaliação']

# Chama a função para encontrar valores únicos e contar a ocorrência de cada valor em cada coluna
unique_values_and_counts(df_avaliacao, columns_to_check)



Contagem de valores na coluna 'id_proponente':
id_proponente
479     13
258     12
427     12
1383    12
715     12
        ..
363      1
1300     1
931      1
1063     1
890      1
Name: count, Length: 1423, dtype: int64
--------------------------------------------

Contagem de valores na coluna 'id_projeto':
id_projeto
1336    16
794     16
277     15
950     15
135     15
        ..
745      1
827      1
1080     1
258      1
1331     1
Name: count, Length: 1423, dtype: int64
--------------------------------------------

Contagem de valores na coluna 'avaliação':
avaliação
4    3154
3    1728
1    1698
2    1473
5    1048
Name: count, dtype: int64
--------------------------------------------


## Pré-processamento

In [31]:
# Renomeando as colunas
df_avaliacao = df_avaliacao.rename(columns={'id_proponente': 'id_evaluator', 'id_projeto': 'id_project', 'avaliação': 'rating'})
df_avaliacao.head()

,id_evaluator,id_project,rating
0,1,1119,4
1,1,300,3
2,1,11,4
3,1,1121,1
4,1,937,4


## Explicações

O dataset de avaliações é exencial para o desenvovimento do projeto LegacyLab. Além de popular o banco de dados vigente na solução, ele apresenta dados sobre os projetos e suas avaliações.


Aqui se encontra a chave do futuro modelo de recomendação.

Um modelo de recomendação baseado em filtragem colaborativa opera **analisando interações históricas entre usuários e itens para prever preferências futuras**. A central ao seu funcionamento está a **matriz de avaliação**, onde usuários são representados nas linhas e itens nas colunas, com cada célula indicando a avaliação de um usuário para um item específico.

Para realizar previsões, o modelo busca usuários similares com base em suas avaliações passadas. Isso é frequentemente feito calculando-se a similaridade entre os padrões de avaliação dos usuários, utilizando métricas como a similaridade de Pearson ou cosseno.

Uma vez identificados usuários similares, o modelo prevê a preferência de um usuário específico por um item não avaliado, com base nas avaliações dos usuários similares. Por exemplo, se um usuário A avaliou um item com alta pontuação e um usuário B é considerado similar a A, o modelo pode prever que B também apreciaria o item.

Esse dataset finalde usuários apresenta 9101 linhas de informação, fornecendo dados sobre:

| Campo         | Descrição                                         |
|---------------|---------------------------------------------------|
| `id_evaluator	`       | ID único do usuário avaliador do projeto                               |
| `id_project`         | ID único do projeto                      |
| `rating`          | Avaliação fornecida pelo usuário (de 1 a 5)                  

# Dataset final

In [32]:
# Combinação dos dataframes df_projeto e df_avaliacao com base no id_project
df_final = pd.merge(df_projeto, df_avaliacao, on='id_project', how='left')
df_final.head()

,id_project,id_owner,project_name,microtheme,macrotheme,status,id_evaluator,rating
0,1,1,Como Manter a Esperança?,Psicologia,Saúde e Bem-estar,Em planejamento,167.0,5.0
1,1,1,Como Manter a Esperança?,Psicologia,Saúde e Bem-estar,Em planejamento,384.0,5.0
2,1,1,Como Manter a Esperança?,Psicologia,Saúde e Bem-estar,Em planejamento,499.0,5.0
3,1,1,Como Manter a Esperança?,Psicologia,Saúde e Bem-estar,Em planejamento,575.0,5.0
4,1,1,Como Manter a Esperança?,Psicologia,Saúde e Bem-estar,Em planejamento,796.0,5.0


In [33]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9102 entries, 0 to 9101
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_project    9102 non-null   int64  
 1   id_owner      9102 non-null   int64  
 2   project_name  9102 non-null   object 
 3   microtheme    9102 non-null   object 
 4   macrotheme    9102 non-null   object 
 5   status        9102 non-null   object 
 6   id_evaluator  9101 non-null   float64
 7   rating        9101 non-null   float64
dtypes: float64(2), int64(2), object(4)
memory usage: 569.0+ KB


## Pré-processamento

### Macrotemas

Para melhorar a experiência do usuário, melhorar o sistema de recomendação e seguir o padrão de macrotemas definidos pela FDC, o LegacyLab decidiu limitar 6 macrotemas:

1. Conservação do Planeta
2. Redução do Impacto Ambiental
3. Bem-Estar, Saúde e Felicidade
5. Produtividade e Competitividade
6. Integridade e Práticas Éticas

In [34]:
# Mapeamento dos macrotemas atuais para as novas categorias
mapeamento_macrotemas = {
    'Saúde e Bem-estar': 'Bem-Estar, Saúde e Felicidade',
    'Educação': 'Produtividade e Competitividade',
    'Energia e Recursos': 'Conservação do Planeta',
    'Ciência e Pesquisa': 'Conservação do Planeta',
    'Sustentabilidade': 'Redução do Impacto Ambiental',
    'Tecnologia e Informação': 'Produtividade e Competitividade',
    'Comunicação e Mídia': 'Integridade e Práticas Éticas',
    'Meio Ambiente': 'Redução do Impacto Ambiental',
    'Tecnologia e Inovação': 'Produtividade e Competitividade',
    'Serviços Sociais': 'Bem-Estar, Saúde e Felicidade',
    'Economia e Finanças': 'Produtividade e Competitividade',
    'Arte e Cultura': 'Integridade e Práticas Éticas',
    'Desenvolvimento Econômico e Empresarial': 'Produtividade e Competitividade',
    'Transporte e Logística': 'Conservação do Planeta',
    'Agricultura e Alimentação': 'Redução do Impacto Ambiental',
    'Engenharia': 'Conservação do Planeta',
    'Desenvolvimento Urbano e Infraestrutura': 'Conservação do Planeta',
    'Direito e Governança': 'Integridade e Práticas Éticas',
    'Geral e Diversificado': 'Produtividade e Competitividade',
    'Saúde e Inovação': 'Bem-Estar, Saúde e Felicidade',
    'Gestão e Recursos Humanos': 'Produtividade e Competitividade',
    'Tecnologia e Comunicação': 'Produtividade e Competitividade',
    'Saúde e Tecnologia': 'Bem-Estar, Saúde e Felicidade',
    'Desenvolvimento Comunitário': 'Redução do Impacto Ambiental',
    'Transporte e Tecnologia': 'Produtividade e Competitividade',
    'Educação e Meio Ambiente': 'Redução do Impacto Ambiental',
    'Tecnologia e Dados': 'Produtividade e Competitividade',
    'Empreendedorismo e Inovação': 'Produtividade e Competitividade',
    'Saúde e Indústria': 'Bem-Estar, Saúde e Felicidade',
    'Desporto e Lazer': 'Bem-Estar, Saúde e Felicidade',
    'Estratégia e Gestão': 'Produtividade e Competitividade',
    'Marketing e Comunicação': 'Integridade e Práticas Éticas',
    'Tecnologia e Educação': 'Produtividade e Competitividade',
    'Tecnologia e Meio Ambiente': 'Redução do Impacto Ambiental',
    'Tecnologia e Saúde': 'Bem-Estar, Saúde e Felicidade',
    'Turismo e Lazer': 'Bem-Estar, Saúde e Felicidade',
    'Saúde e Infraestrutura': 'Bem-Estar, Saúde e Felicidade',
    'Saúde e Pesquisa': 'Bem-Estar, Saúde e Felicidade',
    'Desenvolvimento Internacional': 'Redução do Impacto Ambiental',
    'Turismo e Tecnologia': 'Bem-Estar, Saúde e Felicidade'
}

# Aplicar o mapeamento à coluna 'macrotheme' do DataFrame
df_final['macrotheme'] = df_final['macrotheme'].map(mapeamento_macrotemas)

# Exibir o DataFrame atualizado
print(df_final)

      id_project  id_owner              project_name               microtheme  \
0              1         1  Como Manter a Esperança?               Psicologia   
1              1         1  Como Manter a Esperança?               Psicologia   
2              1         1  Como Manter a Esperança?               Psicologia   
3              1         1  Como Manter a Esperança?               Psicologia   
4              1         1  Como Manter a Esperança?               Psicologia   
...          ...       ...                       ...                      ...   
9097        1424      1424         Projeto Renovação  Desenvolvimento Pessoal   
9098        1424      1424         Projeto Renovação  Desenvolvimento Pessoal   
9099        1424      1424         Projeto Renovação  Desenvolvimento Pessoal   
9100        1424      1424         Projeto Renovação  Desenvolvimento Pessoal   
9101        1424      1424         Projeto Renovação  Desenvolvimento Pessoal   

                           

In [35]:
# Valores por macrotema
df_final['macrotheme'].value_counts()

macrotheme
Produtividade e Competitividade    4050
Bem-Estar, Saúde e Felicidade      1783
Redução do Impacto Ambiental       1642
Conservação do Planeta             1259
Integridade e Práticas Éticas       368
Name: count, dtype: int64

In [36]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9102 entries, 0 to 9101
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_project    9102 non-null   int64  
 1   id_owner      9102 non-null   int64  
 2   project_name  9102 non-null   object 
 3   microtheme    9102 non-null   object 
 4   macrotheme    9102 non-null   object 
 5   status        9102 non-null   object 
 6   id_evaluator  9101 non-null   float64
 7   rating        9101 non-null   float64
dtypes: float64(2), int64(2), object(4)
memory usage: 569.0+ KB


In [37]:
# Encontra as linhas com valores nulos na coluna id_evaluator
linhas_nulas = df_final[df_final['id_evaluator'].isnull()]

# Visualiza as linhas nulas
print(linhas_nulas)

      id_project  id_owner project_name microtheme              macrotheme  \
6591        1034      1034   BioBravura   Biologia  Conservação do Planeta   

               status  id_evaluator  rating  
6591  Em planejamento           NaN     NaN  


In [38]:
# Remove a linha com valores nulos na coluna id_evaluator
df_final = df_final.dropna(subset=['id_evaluator'])

In [39]:
# Transforma em inteiro as colunas de avaliador e de avaliação
df_final['id_evaluator'] = df_final['id_evaluator'].astype(int)
df_final['rating'] = df_final['rating'].astype(int)

<ipython-input-39-370397324c11>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['id_evaluator'] = df_final['id_evaluator'].astype(int)
<ipython-input-39-370397324c11>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['rating'] = df_final['rating'].astype(int)


In [40]:
# Dataset limpo e pronto para começar o modelo de recomendação!
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9101 entries, 0 to 9101
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id_project    9101 non-null   int64 
 1   id_owner      9101 non-null   int64 
 2   project_name  9101 non-null   object
 3   microtheme    9101 non-null   object
 4   macrotheme    9101 non-null   object
 5   status        9101 non-null   object
 6   id_evaluator  9101 non-null   int64 
 7   rating        9101 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 639.9+ KB


### Microtemas

In [41]:
# Entendendo quais microtemas existem por macrotema
def listar_microtemas_por_macrotema(df, macrotema):

    # Filtrar o DataFrame para incluir apenas os dados do macrotema especificado
    macrotema_df = df[df['macrotheme'] == macrotema]

    # Obter a lista de microtemas únicos dentro do macrotema especificado
    microtemas_macrotema = macrotema_df['microtheme'].unique()

    # Exibir a lista de microtemas
    print(f"Microtemas no macrotema '{macrotema}':")
    for microtema in microtemas_macrotema:
        print("-", microtema)
    print("-----------------------------")

# Lista dos 6 macrotemas fornecidos anteriormente
macrotemas = [
    'Conservação do Planeta',
    'Redução do Impacto Ambiental',
    'Bem-Estar, Saúde e Felicidade',
    'DE&I',
    'Produtividade e Competitividade',
    'Integridade e Práticas Éticas'
]

# Iterar sobre cada macrotema e listar os microtemas associados
for macrotema in macrotemas:
    listar_microtemas_por_macrotema(df_final, macrotema)


Microtemas no macrotema 'Conservação do Planeta':
- Energia Renovável
- Genética
- Científico
- Astronomia
- Energia
- Bioquímica
- Física
- Transporte Marítimo
- Biotecnologia
- Urbanismo, Sustentabilidade
- Transporte
- Engenharia de Energia
- Engenharia de Materiais
- Marítimo
- Física Quântica
- Urbanismo
- Desenvolvimento Urbano
- Biologia
- Biologia Evolutiva
- Biologia Celular
- Matemática
- Transporte Urbano
- Biologia Estrutural
- Urbanismo, Tecnologia
- Transporte, Tecnologia
- Transporte Aquático
- Transporte, Social
- Botânica
- Engenharia Biomédica
- Genômica
- Inovações em Energia Limpa
- Acessibilidade Urbana
- Química
- Bioengenharia
- Biologia Molecular
- Astrofísica
- Transporte Marítimo, Tecnologia
- Aviação
- Tecnologia de Energia Renovável
- Biologia Sintética
- Transporte, Energia
- Logística
- Gestão de Frotas
- Bioinformática
- Transporte Ferroviário
- Geologia Planetária
- Geologia
- Ciência, Educação
- Neurociência
- Tecnologia Têxtil
-------------------------

In [42]:
# Função para listar microtemas por macrotema e retornar a lista de microtemas
def listar_microtemas_por_macrotema(df, macrotema):
    # Filtrar o DataFrame para incluir apenas os dados do macrotema especificado
    macrotema_df = df[df['macrotheme'] == macrotema]
    # Obter a lista de microtemas únicos dentro do macrotema especificado
    microtemas_macrotema = macrotema_df['microtheme'].unique()
    return microtemas_macrotema

# Função para encontrar microtemas comuns entre diferentes macrotemas
def encontrar_microtemas_comuns(df, macrotemas):
    microtemas_comuns = {}
    for macrotema in macrotemas:
        microtemas = listar_microtemas_por_macrotema(df, macrotema)
        for microtema in microtemas:
            if microtema not in microtemas_comuns:
                microtemas_comuns[microtema] = [macrotema]
            else:
                microtemas_comuns[microtema].append(macrotema)
    return {microtema: macrotemas for microtema, macrotemas in microtemas_comuns.items() if len(macrotemas) > 1}

# Lista dos 6 macrotemas fornecidos anteriormente
macrotemas = [
    'Conservação do Planeta',
    'Redução do Impacto Ambiental',
    'Bem-Estar, Saúde e Felicidade',
    'DE&I',
    'Produtividade e Competitividade',
    'Integridade e Práticas Éticas'
]

# Encontrar microtemas comuns entre diferentes macrotemas
microtemas_comuns = encontrar_microtemas_comuns(df_final, macrotemas)

# Exibir microtemas comuns e os macrotemas associados
print("Microtemas comuns entre diferentes macrotemas:")
for microtema, macrotemas_associados in microtemas_comuns.items():
    print(f"Microtema: {microtema}, Macrotemas: {macrotemas_associados}")


Microtemas comuns entre diferentes macrotemas:


In [43]:
df_final['microtheme'].value_counts()

microtheme
Educação                                1474
Social                                   905
Tecnologia                               649
Sustentabilidade                         615
Saúde                                    591
                                        ... 
Tecnologia, Transporte                     3
Desenvolvimento de Produto Inclusivo       2
Educação, Negócios                         2
Tecnologia Têxtil                          2
Gestão da Qualidade                        1
Name: count, Length: 203, dtype: int64

# Considerações finais


In [44]:
# Formalizando conforme banco de dados

# Removendo as colunas especificadas
df_final = df_final.drop(columns=['id_owner', 'project_name', 'microtheme', 'macrotheme', 'status'])

# Renomeando as colunas
df_final = df_final.rename(columns={'id_evaluator': 'id_User', 'rating': 'score', 'id_project': 'id_Project'})

# Visualizando o resultado
print(df_final.head())


   id_Project  id_User  score
0           1      167      5
1           1      384      5
2           1      499      5
3           1      575      5
4           1      796      5


### Desbalanceamento: n° de projetos avaliados

Alguns projetos foram avaliados mais vezes que outros. Isso pode gerar desequilíbrio nos dados.

O LegayLab irá realizar uma **amostragem estratificada** durante a **divisão dos dados em conjuntos de treinamento e teste**. Isso garante que as proporções de projetos avaliados sejam mantidas em ambos os conjuntos.

### Desbalanceamento: n° de micro e macrotemas

Assim como os projetos, certos micro e macrotemas têm uma contagem de avaliações muito maior do que outros. Isso pode ser tratado de maneira semelhante à questão dos projetos, garantindo uma representação adequada de todos os micro e macro temas nos conjuntos de treinamento e teste.

### Desbalanceamento: avaliações


Há mais projetos avaliados com uma classificação de 4. Isso pode introduzir viés no modelo. Uma maneira de lidar com isso é usar técnicas como reamostragem, ponderação de classes ou métodos de avaliação de desempenho que levem em consideração o desequilíbrio de classe, como F1-score ou AUC-ROC.

### Desbalanceamento: n° de avaliações por usuário

Serão ponderadas as avaliações dos usuários com base na frequência com que eles avaliam projetos. Por exemplo, se um usuário avalia muitos projetos, suas avaliações irão receber um peso menor para evitar que dominem o processo de treinamento do modelo.

## Exportanto o .CSV

In [45]:
# Transformando o dataset em arquivo .csv
df_final.to_csv('dataset.csv', index=False)

In [46]:
from google.colab import files
# Download automático do arquivo
files.download('dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

fim :)